In [1]:
!pip install langchain==0.2.0 --quiet
!pip install langchain-openai==0.1.7 --quiet
!pip install langchain-community==0.2.0 --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.7/973.7 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 397.1/397.1 kB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.8/311.8 kB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 22.3 MB/s eta 0:00:00


In [27]:
import os
from google.colab import userdata

os.environ['OPENAI_API_KEY'] = userdata.get('OPEN_API_KEY')
os.environ['HUGGINGFACEHUB_API_TOKEN'] = userdata.get('HF_TOKEN')
os.environ['TAVILY_API_KEY'] = userdata.get('TAVILY_API_KEY')
WEATHER_API_KEY = userdata.get('WEATHER_API_KEY')

In [3]:
from langchain_openai import ChatOpenAI

chatgpt = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0)

## Create Tools

Here we create two custom tools which are wrappers on top of the [Tavily API](https://tavily.com/#api) and [WeatherAPI](https://www.weatherapi.com/)

- Simple Web Search tool
- Weather tool

In [25]:
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.tools import tool
import requests
import json

tv_search = TavilySearchResults(max_results=3, search_depth='advanced',
                                max_tokens=10000)

@tool
def search_web(query: str) -> list:
    """Search the web for a query."""
    tavily_tool = TavilySearchResults(max_results=2)
    results = tavily_tool.invoke(query)
    return results

@tool
def get_weather(query: str) -> list:
    """Search weatherapi to get the current weather."""
    base_url = "http://api.weatherapi.com/v1/current.json"
    complete_url = f"{base_url}?key={WEATHER_API_KEY}&q={query}"

    response = requests.get(complete_url)
    data = response.json()
    if data.get("location"):
        return data
    else:
        return "Weather Data Not Found"

## Test Tool Calling with LLM

In [5]:
from langchain_openai import ChatOpenAI

chatgpt = ChatOpenAI(model="gpt-4o", temperature=0)
tools = [search_web, get_weather]

chatgpt_with_tools = chatgpt.bind_tools(tools)

In [6]:
prompt = "who won the champions league in 2024"
response = chatgpt_with_tools.invoke(prompt)
response.tool_calls

[{'name': 'search_web',
  'args': {'query': 'Champions League 2024 winner'},
  'id': 'call_5WVRZTFE5pZnaDpxX8UNhmqF',
  'type': 'tool_call'}]

In [7]:
prompt = "how is the weather in Bangalore today"
response = chatgpt_with_tools.invoke(prompt)
response.tool_calls

[{'name': 'get_weather',
  'args': {'query': 'Bangalore'},
  'id': 'call_c3esXhqfiWs0Vvy8tGVqmsBT',
  'type': 'tool_call'}]

## Build and Test AI Agent

Now that we have defined the tools and the LLM, we can create the agent. We will be using a tool calling agent to bind the tools to the agent with a prompt. We will also add in the capability to store historical conversations as memory

In [8]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

SYS_PROMPT = """Act as a helpful assistant.
                Use the tools at your disposal to perform tasks as needed.
                  - get_weather: whenever user asks get the weather of a place.
                  - search_web: whenever user asks for information on current events or if you don't know the answer.
             """

prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", SYS_PROMPT),
        MessagesPlaceholder(variable_name="history", optional=True),
        ("human", "{query}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

prompt_template.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template="Act as a helpful assistant.\n                Use the tools at your disposal to perform tasks as needed.\n                  - get_weather: whenever user asks get the weather of a place.\n                  - search_web: whenever user asks for information on current events or if you don't know the answer.\n             ")),
 MessagesPlaceholder(variable_name='history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['query'], template='{query}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

Now, we can initalize the agent with the LLM, the prompt, and the tools.

The agent is responsible for taking in input and deciding what actions to take.

REMEMBER the Agent does not execute those actions - that is done by the AgentExecutor

Note that we are passing in the model `chatgpt`, not `chatgpt_with_tools`.

That is because `create_tool_calling_agent` will call `.bind_tools` for us under the hood.

This should ideally be used with an LLM which supports tool \ function calling

In [9]:
from langchain.agents import create_tool_calling_agent

agent = create_tool_calling_agent(chatgpt, tools, prompt_template)
agent

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'query'], optional_variables=['history'], input_types={'history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, partial_variables={'history': []}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template="Act as a helpful assistant.\n 

Finally, we combine the `agent` (the brains) with the `tools` inside the `AgentExecutor` (which will repeatedly call the agent and execute tools).

In [10]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools)
agent_executor

AgentExecutor(agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'query'], optional_variables=['history'], input_types={'history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, partial_variables={'history': []}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_

In [11]:
query = """Tell me who won the champions league in 2024,
            show some detailed information about the match also
        """
response = chatgpt.invoke(query)
response.content

"I'm sorry, but I don't have access to real-time data or events beyond October 2023. Therefore, I cannot provide information about the winner of the UEFA Champions League in 2024 or any details about the match. You may want to check the latest sports news or the official UEFA website for the most up-to-date information."

In [12]:
query = """Tell me who won the champions league in 2024,
            show some detailed information about the match also
        """
response = agent_executor.invoke({"query": query})

In [13]:
response

{'query': 'Tell me who won the champions league in 2024,\n            show some detailed information about the match also\n        ',
 'output': "The 2024 UEFA Champions League final was held at Wembley Stadium in London, England, on June 1, 2024. Real Madrid emerged victorious, defeating Borussia Dortmund with a score of 2-0. This victory allowed Real Madrid to compete against the winners of the 2023–24 UEFA Europa League, Atalanta, in the 2024 UEFA Super Cup.\n\n### Match Details:\n- **Final Score**: Real Madrid 2-0 Borussia Dortmund\n- **Location**: Wembley Stadium, London, England\n- **Date**: June 1, 2024\n- **Player of the Match**: Dani Carvajal (Real Madrid)\n- **Champions League Young Player of the Season**: Jude Bellingham (Real Madrid)\n\nReal Madrid's win in this match added another prestigious title to their illustrious history in European football."}

In [14]:
from IPython.display import display, Markdown

display(Markdown(response['output']))

The 2024 UEFA Champions League final was held at Wembley Stadium in London, England, on June 1, 2024. Real Madrid emerged victorious, defeating Borussia Dortmund with a score of 2-0. This victory allowed Real Madrid to compete against the winners of the 2023–24 UEFA Europa League, Atalanta, in the 2024 UEFA Super Cup.

### Match Details:
- **Final Score**: Real Madrid 2-0 Borussia Dortmund
- **Location**: Wembley Stadium, London, England
- **Date**: June 1, 2024
- **Player of the Match**: Dani Carvajal (Real Madrid)
- **Champions League Young Player of the Season**: Jude Bellingham (Real Madrid)

Real Madrid's win in this match added another prestigious title to their illustrious history in European football.

In [29]:
query = """how is the weather in Bangalore today?
        """
response = agent_executor.invoke({"query": query})
display(Markdown(response['output']))

The weather in Bangalore today is clear with a temperature of 21.1°C (70.0°F). The wind is blowing from the east-southeast at 14.8 kph (9.2 mph), and the humidity is at 60%. There is no precipitation, and the visibility is 6.0 km (3.0 miles).

In [30]:
query = """how is the weather in Dubai today?
        """
response = agent_executor.invoke({"query": query})
display(Markdown(response['output']))

The weather in Dubai today is clear with a temperature of 24.3°C (75.7°F). The wind is blowing from the northeast at 20.9 kph (13.0 mph), and the humidity is at 50%. There is no precipitation, and the visibility is 10 km (6 miles). The current conditions feel like 25.7°C (78.3°F).

In [31]:
query = """which city is hotter?
        """
response = agent_executor.invoke({"query": query})
display(Markdown(response['output']))

Could you please specify the cities you would like to compare in terms of temperature?

The agent is doing pretty well but unfortunately it doesn't remember conversations. Let's now use some memory to store this.

## Build and Test Multi-User Conversational AI Agent

We will now use `SQLChatMessageHistory` which we learnt in the previous module to store separate conversation histories per user or session.

This will help us build a conversational Agentic Chatbot which will be accessed by many users at the same time

In [32]:
# removes the memory database file - usually not needed
# you can run this only when you want to remove ALL conversation histories
# ok if you get rm: cannot remove 'memory.db': No such file or directory  because initially no memory exists
!rm memory.db

rm: cannot remove 'memory.db': No such file or directory


In [33]:
from langchain_community.chat_message_histories import SQLChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

# used to retrieve conversation history from database
# based on a specific user or session ID
def get_session_history_db(session_id):
    return SQLChatMessageHistory(session_id, "sqlite:///memory.db")

# create a conversation chain + agent which can load memory based on specific user or session id
agentic_chatbot = RunnableWithMessageHistory(
    agent_executor,
    get_session_history_db,
    input_messages_key="query",
    history_messages_key="history",
)

# function to call the agent show results per user session
from IPython.display import display, Markdown
def chat_with_agent(prompt: str, session_id: str):
    response = agentic_chatbot.invoke({"query": prompt},
                                      {'configurable': { 'session_id': session_id}})
    display(Markdown(response['output']))

In [34]:
user_id = 'john001'
prompt = "Hi can you tell me who won champions league in 2024"
chat_with_agent(prompt, user_id)

The winner of the 2024 UEFA Champions League was Real Madrid. They defeated Dortmund 2-0 in the final held at Wembley Stadium in London.

In [35]:
prompt = "Tell me more about this event in detail please"
chat_with_agent(prompt, user_id)

Real Madrid won the 2024 UEFA Champions League final by defeating Borussia Dortmund 2-0. The match took place at Wembley Stadium in London. This victory marked Real Madrid's 15th Champions League title, further extending their record as the club with the most titles in the competition's history.

For more detailed highlights and information, you can visit the [UEFA website](https://www.uefa.com/uefachampionsleague/match/2039970--b-dortmund-vs-real-madrid/) or check out the [ESPN match report](https://www.espn.com/soccer/match/_/gameId/702410/real-madrid-borussia-dortmund).

In [36]:
user_id = 'bond007'
prompt = "how is the weather in Bangalore today? Show detailed statistics"
chat_with_agent(prompt, user_id)

The current weather in Bangalore, Karnataka, India is as follows:

- **Temperature**: 21.1°C (70.0°F)
- **Condition**: Clear
- **Wind**: 9.2 mph (14.8 kph) from the East-Southeast (ESE) at 106°
- **Pressure**: 1017.0 mb (30.03 in)
- **Precipitation**: 0.0 mm (0.0 in)
- **Humidity**: 60%
- **Cloud Cover**: 0%
- **Feels Like**: 21.1°C (70.0°F)
- **Wind Chill**: 21.4°C (70.6°F)
- **Heat Index**: 24.3°C (75.7°F)
- **Dew Point**: 10.3°C (50.5°F)
- **Visibility**: 6.0 km (3.0 miles)
- **UV Index**: 0.0
- **Wind Gusts**: 16.1 mph (25.9 kph)

The weather is clear with no cloud cover, and the temperature feels quite comfortable.

In [37]:
user_id = 'bond007'
prompt = "what about Dubai?"
chat_with_agent(prompt, user_id)

The current weather in Dubai, United Arab Emirates is as follows:

- **Temperature**: 24.3°C (75.7°F)
- **Condition**: Clear
- **Wind**: 13.0 mph (20.9 kph) from the Northeast (NE) at 52°
- **Pressure**: 1013.0 mb (29.91 in)
- **Precipitation**: 0.0 mm (0.0 in)
- **Humidity**: 50%
- **Cloud Cover**: 0%
- **Feels Like**: 25.7°C (78.3°F)
- **Wind Chill**: 21.1°C (70.0°F)
- **Heat Index**: 24.5°C (76.0°F)
- **Dew Point**: 13.6°C (56.5°F)
- **Visibility**: 10.0 km (6.0 miles)
- **UV Index**: 0.0
- **Wind Gusts**: 21.8 mph (35.0 kph)

The weather is clear with no cloud cover, and the temperature feels warm and pleasant.

In [38]:
user_id = 'bond007'
prompt = "which city is hotter?"
chat_with_agent(prompt, user_id)

Currently, Dubai is hotter than Bangalore. The temperature in Dubai is 24.3°C (75.7°F), while in Bangalore, it is 21.1°C (70.0°F).